In [3]:
import pandas as pd
from IPython.display import display

file_path = r"C:\Users\brkyi\Desktop\animated visuals datasets\uefa_dataset\UCL_Finals_1955-2023.csv"

for enc in ['utf-8-sig', 'utf-16', 'latin1']:
    try:
        df = pd.read_csv(file_path, encoding=enc)
        print(f"Loaded successfully with encoding: {enc}")
        break
    except:
        pass

df['Season'] = df['Season'].str[:4].astype(int)
df = df[['Season', 'Winners']]

display(df.head())


Loaded successfully with encoding: utf-8-sig


,Season,Winners
0,1955,Real Madrid
1,1956,Real Madrid
2,1957,Real Madrid
3,1958,Real Madrid
4,1959,Real Madrid


In [4]:
import os
from IPython.display import display

logo_dir = r"C:\Users\brkyi\Desktop\animated visuals datasets\uefa_dataset\logos"

def get_logo_path(team):
    for ext in ['.png', '.webp', '.jpg']:
        path = os.path.join(logo_dir, f"{team}{ext}")
        if os.path.exists(path):
            return path
    return None

df["Logo"] = df["Winners"].apply(get_logo_path)
display(df.head())



,Season,Winners,Logo
0,1955,Real Madrid,C:\Users\brkyi\Desktop\animated visuals datase...
1,1956,Real Madrid,C:\Users\brkyi\Desktop\animated visuals datase...
2,1957,Real Madrid,C:\Users\brkyi\Desktop\animated visuals datase...
3,1958,Real Madrid,C:\Users\brkyi\Desktop\animated visuals datase...
4,1959,Real Madrid,C:\Users\brkyi\Desktop\animated visuals datase...


In [21]:
import plotly.express as px
import plotly.io as pio
import pandas as pd
import os
import base64

# --- cumulative titles ---
df_sorted = df.sort_values('Season')
df_counts = (
    df_sorted.groupby(['Season', 'Winners'])
    .size()
    .groupby(level=1)
    .cumsum()
    .reset_index(name='Titles')
)

teams = df_counts['Winners'].unique()
years = sorted(df_counts['Season'].unique())
full = pd.MultiIndex.from_product([years, teams], names=['Season', 'Winners']).to_frame(index=False)
merged = full.merge(df_counts, on=['Season', 'Winners'], how='left').fillna(0)
merged['Titles'] = merged.groupby('Winners')['Titles'].cummax()
merged = merged.merge(df[['Winners', 'Logo']].drop_duplicates(), on='Winners', how='left')

# --- encode logos once ---
encoded_logos = {}
for winner, logo_path in merged[['Winners','Logo']].drop_duplicates().values:
    if logo_path and os.path.exists(logo_path):
        with open(logo_path, "rb") as f:
            encoded_logos[winner] = f"data:image/png;base64,{base64.b64encode(f.read()).decode()}"

# --- create figure ---
fig = px.bar(
    merged,
    y='Winners',
    x='Titles',
    orientation='h',
    color='Winners',
    animation_frame='Season',
    text='Titles',
    title='UEFA Champions League Titles Over Time',
)

fig.update_traces(
    texttemplate='%{text}',
    textposition='inside',
    insidetextanchor='middle'
)

fig.update_layout(
    width=900,
    height=600,
    template='plotly_white',
    title_x=0.5,
    showlegend=False,
    yaxis={'categoryorder':'total ascending'},
    margin=dict(l=150, r=100, t=80, b=80),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# --- hide text labels, use logos as axis markers ---
fig.update_yaxes(showticklabels=False)
y_positions = merged['Winners'].unique().tolist()

fig.update_layout(
    images=[
        dict(
            source=encoded_logos[t],
            xref="paper", yref="y",
            x=-0.05,  # slightly left of the bars
            y=t,
            sizex=0.08, sizey=0.08,
            xanchor="right", yanchor="middle",
            layer="above"
        )
        for t in y_positions if t in encoded_logos
    ]
)

# --- slow animation ---
for f in fig.frames:
    f['layout'] = dict(transition={'duration': 1500})
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1500
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1500

# --- export & open ---
output_path = r"C:\Users\brkyi\Desktop\animated_visual.html"
pio.write_html(fig, file=output_path, auto_open=True)
